In [1]:
import sys, os
sys.path.append('..')
from utils_ft import get_model_path, get_output_dir, setup_env, get_root_path
from scripts.train_pytorch import train_pytorch

# Config
MODEL_TYPE = "llama"
#MODEL_TYPE = "qwen"
hf_token = setup_env()
model_id = get_model_path(MODEL_TYPE)
dataset_file = os.path.join(get_root_path(), "Finetuning", "data", "asp_finetuning_data.jsonl")
output_dir = get_output_dir(MODEL_TYPE, "pytorch")

In [ ]:
train_pytorch(model_id, dataset_file, output_dir, hf_token)